# Use Brute force method to find a combination

In [5]:
import pandas as pd

In [6]:
import numpy as np

In [7]:
import datetime as dt

In [8]:
from collections import Counter

In [9]:
import matplotlib.pyplot as plt

### Import Data

In [10]:
XRP_Increase = pd.read_csv('../XRP_5min_Increase_SmallAmount_5min.csv')
XRP_Decrease = pd.read_csv('../XRP_5min_Decrease_SmallAmount_5min.csv')

In [11]:
original_columns = ['datetime_real','date','close','high','low','open','quoteVolume','volume','weightedAverage']

In [12]:
XRP_Increase.head()

,Unnamed: 0,Unnamed: 0.1,close,date,high,low,open,quoteVolume,volume,weightedAverage,...,Increase_0.1_percent,Increase_0.2_percent,Increase_0.3_percent,Increase_0.4_percent,Increase_0.5_percent,Increase_0.6_percent,Increase_0.8_percent,Increase_1.0_percent,Increase_1.2_percent,Increase_1.5_percent
0,12736,12736,0.069288,1493875200,0.069960,0.069288,0.069960,163.538132,11.431388,0.069900,...,1493875500,1493875500,1493875500,1493875500,1493875500,1493875500,1493875500,1493881200,1493881500,1493881500
1,12737,12737,0.069100,1493875500,0.069949,0.069000,0.069949,113714.544855,7909.709770,0.069558,...,1493875800,1493876700,1493876700,1493876700,1493876700,1493877000,1493881200,1493881200,1493881200,1493881500
2,12738,12738,0.068000,1493875800,0.069100,0.067500,0.069000,220670.240024,15126.256548,0.068547,...,1493876100,1493876100,1493876100,1493876100,1493876100,1493876100,1493876100,1493876100,1493876100,1493876400
3,12739,12739,0.068465,1493876100,0.068465,0.066432,0.067800,271144.858354,18298.825581,0.067487,...,1493876400,1493876400,1493876400,1493876400,1493876400,1493876400,1493876400,1493876400,1493876400,1493876400
4,12740,12740,0.068646,1493876400,0.068646,0.067517,0.067802,595249.214904,40678.358104,0.068338,...,1493876700,1493876700,1493876700,1493876700,1493876700,1493876700,1493876700,1493876700,1493876700,1493876700


In [13]:
np.array( [1,2,3,4,5,6])*0.1

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6])

In [14]:
# for n in np.array( [1,2,3,4,5,6,8,10])*0.1:
#     XRP_Increase['shift_open_{}'.format(n)] = XRP_Increase['Increase_{}_percent'.format(n)].shift(1)

for 0.6% incraese:

how many continous increase are there?

In [16]:
# XRP_Increase['diff_0.6'] = XRP_Increase['shift_open_0.6000000000000001'] - XRP_Increase['Increase_0.6000000000000001_percent']

In [18]:
#(XRP_Increase[XRP_Increase['diff_0.6'] != 0]['diff_0.6'].value_counts())/(len(XRP_Increase))

In [15]:
n = 6*0.1

## Adjust for waiting

In [109]:
for col in ['Unnamed: 0','Unnamed: 0.1']:
    try:
        del XRP_Increase[col]
    except:
        None

In [110]:
XRP_Analysis_table = XRP_Increase

In [123]:
def brute_force_finding(drop_limit, stop_loss_limit, sell_limit, RH_time, data_to_use, timing_stop = 31536000):
    """     
    inputs:
            drop_limit
            stop_loss_limit
            sell_limit
            RH_time
            XRP_Analysis_table
            
            timing_stop (options), defult = 31536000 ( 1 year time )
            
    """
    
    time_event = {}
    starting_asset = 50
    bought_crypto = False
    gold_available = starting_asset 
    XRP_amount = 0


#     drop_limit = 0.005 # used mean on this side, actual 0.004686, but decreased abit for optimistic view 
#     stop_loss_limit = 0.01
#     sell_limit = 0.006
#     RH_time = '2H_RA_on_Close'


    "inputs"
    drop_limit         = drop_limit
    stop_loss_limit    = stop_loss_limit
    sell_limit         = sell_limit
    RH_time            = RH_time
    XRP_Analysis_table = data_to_use
    timing_stop        = timing_stop
    
    
    if RH_time not in data_to_use.columns:
        print("check columns of data imported, column of RH attempted : {}".format(RH_time))
        return "Wrong data input"
    

    while gold_available >= 0:

        #for ind, df in XRP_Analysis_table[XRP_Analysis_table['datetime_real'] > '2017-07-07'].iterrows():
        for ind, df in XRP_Analysis_table.iterrows():
    #         print(df['date'])

            if bought_crypto != True: # then looking to buy at drop limit or lower
                if ( df[RH_time] - df['low'] ) / df[RH_time] >= drop_limit:
                    "then buy crypto"

                    ## might add here a waiting time!!! ###

                    XRP_amount += (gold_available/df['low']) * 0.998 # due to 0.2% fee

                    gold_available -= gold_available

                    time_event.update({df['date']  : ( gold_available, XRP_amount , 'brought' , df['low'])})

                    last_xrp_traction_price =  df['low']

                    bought_crypto = True
                    
                    time_brought = df['date'] 
    #                 print('last_xrp_traction_price ', last_xrp_traction_price)

            else:
                if ( df[RH_time] - df['low'] ) / df[RH_time] >= stop_loss_limit: # then hit stop loss, force sell
#                     print('stoppppppppppppppppppppppppp')
                    gold_available += XRP_amount * df['low'] * 0.998  # sell at low price

                    XRP_amount -= XRP_amount

                    time_event.update({df['date']  : ( gold_available, XRP_amount , 'stop_loss_hit' ,  df['low'])})

                    bought_crypto = False

                # Time limit, might not need
#                 " other limits needed - such as RA "
                elif df['date'] - time_brought >= timing_stop: # then time limit reached, sell

                    #print(last_xrp_traction_price)

                    gold_available += XRP_amount * df['high'] * 0.998  # sell at high price

                    XRP_amount -= XRP_amount

                    time_event.update({df['date']  : ( gold_available, XRP_amount , 'time_stop' , df['close'])})

                    bought_crypto = False

                elif (df['high'] - last_xrp_traction_price)/last_xrp_traction_price >= sell_limit: # then hit target price, sell for profit

#                     print(last_xrp_traction_price)

                    gold_available += XRP_amount * df['high'] * 0.998  # sell at high price

                    XRP_amount -= XRP_amount

                    time_event.update({df['date']  : ( gold_available, XRP_amount , 'sell_profit' , df['high'])})

                    bought_crypto = False                    
                    
                    

        break

    else:
        print('Negative asset: bankrupt  ->>>   ',  gold_available)
        
    transation_log = pd.DataFrame.from_dict(time_event, orient='index', columns= ['Gold', 'XRP_Amount', 'action', 'price_action_happened'])
    transation_log['Gold'] = round(transation_log['Gold'], 2)
    transation_log['XRP_Amount'] = round(transation_log['XRP_Amount'], 4)
    
    print('end Gold amount ' , transation_log.iloc[-1]['Gold'])
    return (transation_log.iloc[-1]['Gold'] , transation_log)

In [124]:
XRP_Analysis_table['20min_RA_on_Close'] = XRP_Analysis_table['close'].rolling(4).mean().shift(1)
XRP_Analysis_table['1H_RA_on_Close'] = XRP_Analysis_table['close'].rolling(12).mean().shift(1)
XRP_Analysis_table['2H_RA_on_Close'] = XRP_Analysis_table['close'].rolling(24).mean().shift(1)
XRP_Analysis_table['4H_RA_on_Close'] = XRP_Analysis_table['close'].rolling(48).mean().shift(1)
XRP_Analysis_table['12H_RA_on_Close'] = XRP_Analysis_table['close'].rolling(144).mean().shift(1)

In [125]:
ra_list            =    ['20min_RA_on_Close', '1H_RA_on_Close', '2H_RA_on_Close', '4H_RA_on_Close', '12H_RA_on_Close']
buy_in_list        =    np.arange(0.002,0.022, 0.002)
stop_loss_list     =    np.arange(0.01,0.03, 0.002)
time_waiting_list  =    [2, 5, 8, 12, 16, 24] # in hours
sell_limit_list    =    np.arange(0.002,0.032, 0.002)


In [126]:
for time in dates_cors.items():
    print(time[0], time[1])

1507374600 1511274300
1512774600 1515174000
1515174300 1518174000
1547873100 1550872800


In [130]:
event_dictionary = []


for buy_in_input in buy_in_list:
    
    for stop_loss_input in stop_loss_list:
        
        for sell_limit_input in sell_limit_list:
            
            for time_waiting_input in time_waiting_list:
                
                for ra in ra_list:
                    
                    for time in dates_cors.items():
                        
                        print('buy_in at:', buy_in_input,
                              'stop_loss at:', stop_loss_input,
                              'sell_limit at:', sell_limit_input,
                              'time_waiting at:', time_waiting_input,
                              'ra at:', ra,
                              'time at:', time[0], '-' , time[1])
                        
                        temp = brute_force_finding(drop_limit      =  buy_in_input,
                                            stop_loss_limit =  stop_loss_input,
                                            sell_limit      =  sell_limit_input,
                                            RH_time         =  ra,
                                            data_to_use     =  XRP_Analysis_table[(XRP_Analysis_table['date'] >= time[0] ) &
                                                                                  (XRP_Analysis_table['date'] <= time[1] )],
                                            
                                            timing_stop     =  time_waiting_input*60*60
                                           )
                        
                        event_dictionary.append((buy_in_input,
                                                 stop_loss_input,
                                                 sell_limit_input,
                                                 ra,
                                                 time[0],
                                                 time[1],
                                                 time_waiting_input,
                                                 temp
                                                ))




buy_in at: 0.002 stop_loss at: 0.01 sell_limit at: 0.002 time_waiting at: 2 ra at: 20min_RA_on_Close time at: 1507374600 - 1511274300
end Gold amount  0.0
buy_in at: 0.002 stop_loss at: 0.01 sell_limit at: 0.002 time_waiting at: 2 ra at: 20min_RA_on_Close time at: 1512774600 - 1515174000
end Gold amount  0.0
buy_in at: 0.002 stop_loss at: 0.01 sell_limit at: 0.002 time_waiting at: 2 ra at: 20min_RA_on_Close time at: 1515174300 - 1518174000


KeyboardInterrupt: 

In [131]:
event_dictionary

[(0.002,
  0.01,
  0.002,
  '20min_RA_on_Close',
  1507374600,
  1511274300,
  2,
  (0.0,               Gold  XRP_Amount         action  price_action_happened
   1507374600    0.00    214.4419        brought               0.232697
   1507376100   50.11      0.0000    sell_profit               0.234144
   1507376700    0.00    215.0434        brought               0.232556
   1507378500   50.17      0.0000    sell_profit               0.233758
   1507380900    0.00    214.5986        brought               0.233306
   1507381200   50.12      0.0000    sell_profit               0.234000
   1507382700    0.00    213.7410        brought               0.234000
   1507383000   50.05      0.0000    sell_profit               0.234613
   1507383300    0.00    213.4441        brought               0.234000
   1507383600   49.98      0.0000    sell_profit               0.234614
   1507384200    0.00    213.3261        brought               0.233806
   1507384500   49.91      0.0000    sell_profit 

In [91]:
print(brute_force_finding.__doc__)

     
    inputs:
            drop_limit
            stop_loss_limit
            sell_limit
            RH_time
            XRP_Analysis_table
            
            timing_stop (options), defult = 31536000 ( 1 year time )
            
    
